In [ ]:
push!(LOAD_PATH, homedir()*"/Projects/QuIPS/src");

# QuIPS: Quantum Information Processing System

This notebook contains a brief introduction to QuIPS, a pure Julia implementation of a Quantum Abstract Machine. Let's start by bringing QuIPS into scope, and defining the type we will be working with.

In [ ]:
using QuIPS

C = Complex{Float32}; # makes life easier

## Overview

QuIPS comes with the following:

* SuperTypes
    * Operator
* Types
    * Gate
    * Control
    * Measurement
* Constants
    * GATES
    * PARAM_GATES
* Functions
    * run!
    * step!
    * reset!
    * operate!
    * tensor
    * $\otimes$
    * ^
    * show



## Operators

There are 3 types of operators
* gates 
* controlled gates 
* measurement

Operators here are, in computer science terms, closures, where they are a data structure we can create that then become functions that can act on the wavefunction of the quantum virtual machine.

For example to construct a CNOT, or CX, gate on qubits 2 and 1, do:

In [ ]:
CX̂ = Control((:CX, (1, 2)));

This implementation defines the initial product state wavefunction to be:

$$
|\psi\rangle = |0\rangle_1 \otimes |0\rangle_2 \otimes \cdots \otimes |0\rangle_N = \bigotimes_{i=1}^N|0\rangle_i
$$

For example lets try operating our CX gate on the state $|\psi\rangle = |1\rangle \otimes |0\rangle = |10\rangle$

In [ ]:
ψ = [0,1] ⊗ [1,0]

In [ ]:
CX̂.gate.U # the single qubit X gate is the kernel of this operation

In [ ]:
N = 2 # number of qubits, we need to give this to the gate
ψ = Complex{Float32}.(ψ)

CX̂(ψ, N)

In [ ]:
tag = (:CCNOT, (3, 1, 2))

CCNOT = Control(tag)

CCNOT.gate.U # has the same kernel

In [ ]:
ψ′ = [0,1] ⊗ [0,1] ⊗ [0,1]

In [ ]:
Int.(CCNOT(C.(ψ′), 3))

In [ ]:
CCPHASE(θ, i, j, k) = Control((:CCPHASE, Float32(θ), (i, j, k)))

G = CCPHASE(π \ 4, 3, 1, 2)

G(C.(ψ′), 3)

In [ ]:
show(CCNOT)

In [ ]:
R̂ₓ(γ, k) = Gate((:RX, Float32(γ), k))

R̂ = R̂ₓ(π/4, 1)

R̂.U

In [ ]:
PARAM_GATES

In [ ]:
GATES

## Tensor

Now that was easy since the target qubit (2), was one above the control qubit (1), and CX can be represented as:

$$
C\hat{X} = |0\rangle\langle0| \otimes \hat{I} + |1\rangle\langle1| \otimes \hat{X} = \begin{pmatrix}
1 &0 &0 &0 \\
0 &1 &0 &0 \\
0 &0 &0 &1 \\
0 &0 &1 &0
\end{pmatrix}
$$

If we want to do something like $CC\hat{H}(5,2,7)$ or something else crazy like that we need to work harder to get the numbers to work out correctly. 

A necessity of a QVM is the ability to tensor, or lift, operators up to their correct representation.  Say we have $\hat{Z}_k$ acting on qubit $k$, if we have $N$ qubits then the correct representation is:

$$
\tilde{Z}_k = \hat{I}_1 \otimes \hat{I}_2 \otimes \cdots \otimes \hat{Z}_k \otimes \cdots \otimes \hat{I}_N
$$

From now on, an operator with a tilde has been lifted.

Lets see what this looks like with $\hat{Y}_2$ acting on the middle of 3 qubits.

$$
\tilde{Y} = \hat{I}_1 \otimes \hat{Y}_2 \otimes \hat{I}_3
$$

In [ ]:
N = 3
Y = Complex{Float32}.(GATES[:Y])

In [ ]:
Ỹ = tensor(Y, 2, N)

Int.(imag(Ỹ))

We can accomplish gates like CCX(i, j, k) for arbitrary $i, j$ & $k$ by means of permuting the wavefunction in a way that places the target qubit, $Q_k$, adjacent to control qubits, $C_i$ & $C_j$, and in the correct order for the standard Control operator representation to be applicable.  If $Q_k \in \mathscr{H}_k$, where $\mathscr{H}_k$ is the Hilbert space corresponding to qubit $Q_k$. We can then define a map:

$$
\tau : \mathscr{H}_j \otimes \mathscr{H}_k \to \mathscr{H}_k \otimes \mathscr{H}_j
$$

The kernel of $\tau$ is the SWAP gate,

$$
\text{SWAP} = \begin{pmatrix}
1 &0 &0 &0 \\
0 &0 &1 &0 \\
0 &1 &0 &0 \\
0 &0 &0 &1
\end{pmatrix}
$$

We can use $\tau$ as a transformation operator mapping $Q_j \to Q_{j+1}$, where

$$
\tau_{i,N} := \text{tensor}(\text{SWAP}, i + 1, N)
$$

and this method of tensor just lifts the operator in the correct spot.  We can then define an operator,

$$
\sigma_{j,k} : \mathscr{H}_k \to \mathscr{H}_j,
$$

where, using the rightward product,

$$
\sigma_{j,k,N} = \prod_{i=k}^{j-1}\tau_{k+j-i-1, N}
$$

We are now able to move qubits around, allowing for arbitrary 2 & 3 qubit control gates. so we can transform any operator into a corresponding operator with an easily accessible representation. Of course $\sigma$ is unitary so,

$$
U' = \sigma^{\dagger}U\sigma
$$

and since $\sigma$ is just a product of SWAP gates, its inverse is just the reverse product:

$$
\sigma_{j,k,N}^{\dagger} = \prod_{i=j}^{k}\tau_{k+j-i, N}
$$

thus, for every operator on multiple qubits, we must check and adjust the targets, and for 3 qubit gates this gets a bit trickier.  Regardless, we are able to map every operator in the QCircuit, to a transformed representation we can do a calculation with. fs

## QVM

The QVM module provides a framework to write and run quantum information programs, or quips.  

A quip is a list of operator tags, or tuples of information about the operator, which will get compiled into Operator instances in a quantum circuit, or QCircuit.

Here is an example of a simple quip to entangle and measure 2 qubits:

In [ ]:
quip = [
    (:H, 2),
    (:CX, (2, 1)),
    (:MEASURE, 2),
    (:MEASURE, 1)
];

We can create a QCircuit with a quip,

In [ ]:
N = 2

QC = QCircuit(quip, 2);

and then run our virtual quantum computer,

In [ ]:
run!(QC)

In [ ]:
QC.out

In [ ]:
for i = 1:10
    reset!(QC)
    run!(QC, verbose)
    println(QC.out)
end

and we see entanglement in correlation.